<a href="https://colab.research.google.com/github/samuel0711/image-anomaly-detection/blob/main/inicio_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import io
import sys
import os
import gc
import shutil
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as skio
from skimage.transform import resize
import torch
from torch import nn #neural networks



if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir("/content/drive/My Drive/Colab Notebooks")
    sys.path.append('/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


*   Peds1 tem 34 videos de treino e 36 de teste
*   Peds2 tem 16 videos de treino e 12 de teste (associadas a eventos anômalos)
*   Amostras de treino somente tem frames normais
*   Cada amostra de teste contém pelo menos alguns frames anômalo.
*   10 clips de teste de Peds1 e 12 de Peds2 são providenciados com 'Pixel Level Groundtruth Masks' com sufixo '_gt'


In [2]:
path = os.path.join(os.path.abspath(os.getcwd()), "UCSD_Anomaly_Dataset.v1p2/") #UCSDped1/Test/Test014/

tns = []


subfolders = [os.path.join(path, subfolder) for subfolder in os.listdir(path) if os.path.isdir(os.path.join(path, subfolder))]


#substituido arquivo que deu erro Test017/142
def extract_datasets(subfolders):
  tns = []
  limit = 4000
  i = 0
  reboot = True

  for subfolder in subfolders:
    #print('subfolder: '+subfolder)
    for root, idx, filenames in os.walk(subfolder):
      #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
      if "Train" in root:
        #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
        for filename in filenames: 
            if ".tif" in filename and i < limit:
              img = skio.imread(os.path.join(root, filename))
              tns.append(resize(img, (120,120), preserve_range = True))
              # print('root: ',root, filename)
              # print('\n',img.shape)
              i = i + 1
            elif  i >= limit and i < (limit*2):
              if reboot:
                np.save('train_dataset', np.array(tns, dtype = float))
                tns = []
                reboot = False
              elif ".tif" in filename:
                img = skio.imread(os.path.join(root, filename))
                tns.append(resize(img, (120,120), preserve_range = True))
                # print('root: ',root, filename)
                # print('\n',img.shape)
                i = i + 1

  
  np.save('isolation_train_dataset', np.array(tns, dtype = float))

  print(np.asarray(tns).shape)
  print(np.asarray(tns[0]).shape)

  tns = []
  limit = 1000
  i = 0
  reboot = True

  for subfolder in subfolders:
      #print('subfolder: '+subfolder)
      for root, idx, filenames in os.walk(subfolder):
        #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
        if "Test" in root:
          #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
          for filename in filenames: 
              if ".tif" in filename and i < limit:
                img = skio.imread(os.path.join(root, filename))
                tns.append(resize(img, (120,120), preserve_range = True))
                # print('root: ',root, filename)
                # print('\n',img.shape)
                i = i + 1
              elif  i >= limit and i < (limit*2):
                if reboot:
                  np.save('test_dataset', np.array(tns, dtype = float))
                  tns = []
                  reboot = False
                elif ".tif" in filename:
                  img = skio.imread(os.path.join(root, filename))
                  tns.append(resize(img, (120,120), preserve_range = True))
                  # print('root: ',root, filename)
                  # print('\n',img.shape)
                  i = i + 1

  
  np.save('isolation_test_dataset', np.array(tns, dtype = float))

  print(np.asarray(tns).shape)
  print(np.asarray(tns[0]).shape)
  


extract_datasets(subfolders)

print(type(tns))
#print(tns)

(4000, 120, 120)
(120, 120)
(1000, 120, 120)
(120, 120)
<class 'list'>
